In [1]:
import numpy as np
import pandas as pd
from matplotlib import gridspec
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d, interp2d
from glob import glob
import seaborn as sea
import os

sea.set(style='white')
sea.set(style='ticks')
sea.set_style({'xtick.direct'
               'ion': 'in','xtick.top':True,'xtick.minor.visible': True,
               'ytick.direction': "in",'ytick.right': True,'ytick.minor.visible': True})

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [2]:
###### REPLACE WITH YOUR FILE LOCATION #######
gnd_EZ = fits.open('/Users/rosaliaobrien/research/website/eazy_catalogs/goodsn_3dhst.v4.4.data.fits')
gsd_EZ = fits.open('/Users/rosaliaobrien/research/website/eazy_catalogs/goodss_3dhst.v4.4.data.fits')

gnd_EZ_cat = Table.read(fits.open('/Users/rosaliaobrien/research/website/eazy_catalogs/goodsn_3dhst.v4.4.zout.fits'),
                    format = 'fits').to_pandas()
gsd_EZ_cat = Table.read(fits.open('/Users/rosaliaobrien/research/website/eazy_catalogs/goodss_3dhst.v4.4.zout.fits'),
                    format = 'fits').to_pandas()

gnd_cat = Table.read('/Users/rosaliaobrien/research/website/catalogs/goodsn_3dhst.v4.4.cat',
                    format = 'ascii').to_pandas()
gsd_cat = Table.read('/Users/rosaliaobrien/research/website/catalogs/goodss_3dhst.v4.4.cat',
                    format = 'ascii').to_pandas()

masterlist = pd.read_pickle('/Users/rosaliaobrien/research/website/eazy_catalogs/templates/master_template_list.pkl')

In [3]:
def Phot_get(field, galaxy_id, masterlist):
#     cat = catalog[catalog.id == galaxy_id]

    if field[1] == 'N':
        pre= 'N_'
        cat = gnd_cat[gnd_cat.id == galaxy_id]

    else:
        pre= 'S_'
        cat = gsd_cat[gsd_cat.id == galaxy_id]

    eff_wv = []
    phot_fl = []
    phot_er = []
    phot_num = []

    for i in cat.keys():
        if i[0:2] == 'f_':
            Clam = 3E18 / masterlist.eff_wv[masterlist.tmp_name == pre + i].values[0] **2 * 10**((-1.1)/2.5-29)
            if cat[i].values[0] > -99.0:
                eff_wv.append(masterlist.eff_wv[masterlist.tmp_name == pre + i].values[0])
                phot_fl.append(cat[i].values[0]*Clam)
                phot_num.append(masterlist.tmp_num[masterlist.tmp_name == pre + i].values[0])
        if i[0:2] == 'e_':
            if cat[i].values[0] > -99.0:
                phot_er.append(cat[i].values[0]*Clam)

    return np.array([eff_wv, phot_fl, phot_er])

def read_templates(wave, temps):
    templates = []

    for temp in temps:
        templ = Template(wave, temp)
        templ.set_fnu()
        templates.append(templ)

    return templates

class Template():
    def __init__(self,tempwv, tempflux):
        self.wave = tempwv
        self.flux = tempflux
        self.flux_fnu = None
        self.set_fnu()

    def set_fnu(self):
        self.flux_fnu = self.flux * self.wave**2 / 3.e18


def full_sed(z, coeffs, templates):
    TEMP =read_templates(template_wv,templates)                      
    NTEMP = len(TEMP)
    
    import astropy.units as u

    templ = TEMP
    tempflux = np.zeros((NTEMP, templ[0].wave.shape[0]))
    for i in range(NTEMP):
        tempflux[i, :] = templ[i].flux_fnu

    templz = templ[0].wave*(1+z)

    templf = np.dot(coeffs, tempflux)
    return templz, templf

In [4]:
templates = gnd_EZ['TEMPF'].data
template_wv =gnd_EZ['TEMPL'].data

In [5]:
def Generate_plot(Field, galaxy_ID):
    
    if Field[1] == 'N':
        coeffs = gnd_EZ['COEFFS'].data[galaxy_ID - 1]
        zgrid = gnd_EZ['ZGRID'].data 
        zchi2 = gnd_EZ['CHI2'].data[galaxy_ID - 1]
        zbest = gnd_EZ_cat.query('id == {0}'.format(galaxy_ID)).z500.values[0]
    
    else:
        coeffs = gsd_EZ['COEFFS'].data[galaxy_ID - 1]
        zgrid = gsd_EZ['ZGRID'].data 
        zchi2 = gsd_EZ['CHI2'].data[galaxy_ID - 1]
        zbest = gsd_EZ_cat.query('id == {0}'.format(galaxy_ID)).z500.values[0]

    ####### get photometry
    effwv, phot, phot_er = Phot_get(Field, galaxy_ID, masterlist)
    
    ####### generate SED
    wv,fl = full_sed(zbest, coeffs, templates)
    fnu_factor = 10**(-0.4*(25.0+48.6))
    flam_spec = 3.e18/wv**2*1E19

    ####### set up grid
    gs = gridspec.GridSpec(1,4, wspace=.3)

    plt.figure(figsize = [20,4])
    
    ####### make flam plot
    plt.subplot(gs[0])
    plt.plot(np.log10(wv*1E-4),fl*fnu_factor*flam_spec, alpha=0.5, label ='z = {0:1.3f}'.format(zbest))
    plt.errorbar(np.log10(effwv*1E-4), phot*1E19, phot_er*1E19, color='r', marker = 'o', linestyle = 'none', 
                 markersize = 7)
    plt.xticks(np.log10([0.5, 1, 2.5, 5.0]),[0.5, 1, 2.5, 5.0])
    plt.legend(fontsize = 10, frameon = True)
    plt.xlabel('Observed Wavelength ($\mu$m)', fontsize=17)
    plt.ylabel('F$_\lambda$ ($10^{-19}$ $erg/s/cm^{2}/\AA $)', fontsize=17)
    plt.tick_params(axis='both', which='major', labelsize=13)
    plt.xlim(np.log10(0.3),np.log10(9))

    ####### make fnu plot
    iscale = interp1d(wv,flam_spec)(effwv)
    fnu = phot*1E19 / iscale
    dfnu = phot_er*1E19/ iscale    
    
    plt.subplot(gs[1])
    plt.plot(np.log10(wv*1E-4),np.log10(fl*fnu_factor), alpha=0.5)
    plt.errorbar(np.log10(effwv*1E-4), np.log10(fnu), dfnu / (fnu * np.log(10)), color='r', marker = 'o', 
                 linestyle = 'none', markersize = 7)
    plt.xticks(np.log10([0.5, 1, 2.5, 5.0]),[0.5, 1, 2.5, 5.0])

    plt.xlabel('Observed Wavelength ($\mu$m)', fontsize=17)
    plt.ylabel('$\log(F_\\nu)$ $\log(10^{23}$ Jy)', fontsize=17)
    plt.tick_params(axis='both', which='major', labelsize=13)
    plt.xlim(np.log10(0.3),np.log10(9))
    plt.ylim(min(np.log10(fnu))*1.02 , max(np.log10(fnu)) * 0.98)

    ####### make chi2 plot
    
    plt.subplot(gs[2])
    plt.plot(zgrid, zchi2)
    plt.xlabel('Redshift (z)', fontsize=17)
    plt.ylabel('$\chi^2$', fontsize=17)
    plt.tick_params(axis='both', which='major', labelsize=12)

    ####### make P(z) plot
    plt.subplot(gs[3])
    P = np.exp( - np.array(zchi2).astype(np.float128) / 2)
    P /= np.trapz(P,zgrid)

    plt.plot(zgrid, P)
    plt.xlabel('Redshift (z)', fontsize=17)
    plt.ylabel('P(z)', fontsize=17)
    plt.tick_params(axis='both', which='major', labelsize=12)
    #NOTE: they are currently all saving in GOODS_South folder (change after error is fixed)
    plt.savefig('/Users/rosaliaobrien/research/website/eazy_plots/{0}_{1}_EZ.png'.format(Field, galaxy_ID), 
                bbox_inches='tight')

In [7]:
#turn off interactive mode so a million plots dont display while running code
plt.ioff()

subfields = ['GN2']

for j in subfields:
    df=pd.read_csv('/Users/rosaliaobrien/research/website/website_data/'+j+'.cat', sep=' ')
    gid=df['ids']
    
    #pass objects that give error (have values that are NaN or Inf)
    for i in gid:
        print('Generating plot '+str(j)+'_'+str(i))
        try:
            Generate_plot(j, i)
        except:
            pass

Generating plot GN2_8140
Generating plot GN2_8300
Generating plot GN2_8762
Generating plot GN2_9030
Generating plot GN2_9083
Generating plot GN2_9128
Generating plot GN2_9178


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered in log10
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in log10


Generating plot GN2_9211
Generating plot GN2_9283
Generating plot GN2_9388
Generating plot GN2_9396
Generating plot GN2_9421
Generating plot GN2_9442
Generating plot GN2_9458
Generating plot GN2_9517
Generating plot GN2_9569
Generating plot GN2_9594
Generating plot GN2_9670
Generating plot GN2_9706
Generating plot GN2_9717
Generating plot GN2_9748


/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:523: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Generating plot GN2_9869
Generating plot GN2_9926
Generating plot GN2_9948
Generating plot GN2_10101
Generating plot GN2_10106
Generating plot GN2_10153
Generating plot GN2_10172
Generating plot GN2_10202
Generating plot GN2_10205
Generating plot GN2_10209
Generating plot GN2_10219
Generating plot GN2_10284
Generating plot GN2_10304
Generating plot GN2_10336
Generating plot GN2_10375
Generating plot GN2_10389
Generating plot GN2_10393
Generating plot GN2_10470
Generating plot GN2_10493
Generating plot GN2_10512
Generating plot GN2_10528
Generating plot GN2_10566
Generating plot GN2_10619
Generating plot GN2_10639
Generating plot GN2_10653
Generating plot GN2_10657
Generating plot GN2_10664
Generating plot GN2_10678


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: RuntimeWarning: divide by zero encountered in log10


Generating plot GN2_10723
Generating plot GN2_10728
Generating plot GN2_10752
Generating plot GN2_10773
Generating plot GN2_10886
Generating plot GN2_10921
Generating plot GN2_10926
Generating plot GN2_11011
Generating plot GN2_11020
Generating plot GN2_11066
Generating plot GN2_11115
Generating plot GN2_11128
Generating plot GN2_11152
Generating plot GN2_11159
Generating plot GN2_11160
Generating plot GN2_11201
Generating plot GN2_11228
Generating plot GN2_11288
Generating plot GN2_11327
Generating plot GN2_11334
Generating plot GN2_11339
Generating plot GN2_11344
Generating plot GN2_11346
Generating plot GN2_11381
Generating plot GN2_11393
Generating plot GN2_11429
Generating plot GN2_11431
Generating plot GN2_11443
Generating plot GN2_11487
Generating plot GN2_11502
Generating plot GN2_11514
Generating plot GN2_11526
Generating plot GN2_11542
Generating plot GN2_11561
Generating plot GN2_11743
Generating plot GN2_11768
Generating plot GN2_11784
Generating plot GN2_11785
Generating p

Generating plot GN2_17436
Generating plot GN2_17437
Generating plot GN2_17477
Generating plot GN2_17481
Generating plot GN2_17489
Generating plot GN2_17502
Generating plot GN2_17515
Generating plot GN2_17530
Generating plot GN2_17553
Generating plot GN2_17579
Generating plot GN2_17589
Generating plot GN2_17626
Generating plot GN2_17635
Generating plot GN2_17646
Generating plot GN2_17667
Generating plot GN2_17708
Generating plot GN2_17712
Generating plot GN2_17714
Generating plot GN2_17727
Generating plot GN2_17748
Generating plot GN2_17760
Generating plot GN2_17782
Generating plot GN2_17792
Generating plot GN2_17799
Generating plot GN2_17802
Generating plot GN2_17822
Generating plot GN2_17829
Generating plot GN2_17861
Generating plot GN2_17862
Generating plot GN2_17886
Generating plot GN2_17891
Generating plot GN2_17898
Generating plot GN2_17940
Generating plot GN2_17942
Generating plot GN2_17963
Generating plot GN2_17977
Generating plot GN2_17986
Generating plot GN2_18022
Generating p

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: RuntimeWarning: divide by zero encountered in log10
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: RuntimeWarning: divide by zero encountered in true_divide
/anaconda3/lib/python3.6/site-packages/matplotlib/axes/_axes.py:3030: RuntimeWarning: invalid value encountered in double_scalars
  high = [thisx + thiserr for (thisx, thiserr)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: RuntimeWarning: divide by zero encountered in log10


Generating plot GN2_20006
Generating plot GN2_20036
Generating plot GN2_20047
Generating plot GN2_20050
Generating plot GN2_20051
Generating plot GN2_20052
Generating plot GN2_20073
Generating plot GN2_20093
Generating plot GN2_20120
Generating plot GN2_20124
Generating plot GN2_20130
Generating plot GN2_20177
Generating plot GN2_20189
Generating plot GN2_20193
Generating plot GN2_20228
Generating plot GN2_20252
Generating plot GN2_20259
Generating plot GN2_20298
Generating plot GN2_20312
Generating plot GN2_20320
Generating plot GN2_20325
Generating plot GN2_20334
Generating plot GN2_20344
Generating plot GN2_20347
Generating plot GN2_20351
Generating plot GN2_20388
Generating plot GN2_20399
Generating plot GN2_20426
Generating plot GN2_20427
Generating plot GN2_20460
Generating plot GN2_20531
Generating plot GN2_20605
Generating plot GN2_20698
Generating plot GN2_20701
Generating plot GN2_20705
Generating plot GN2_20738
Generating plot GN2_20755
Generating plot GN2_20766
Generating p